<a href="https://colab.research.google.com/github/ANGB022210151/AquacultureProject/blob/main/isolation_forest_Step3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# 1. Load your data into a Pandas DataFrame from the CSV file
df = pd.read_csv('/content/full_dataset_with_features.csv')

# Display the first 5 rows to verify the data is loaded correctly
display(df.head())

,time,temperature_cleaned,tds_cleaned,turbidity_cleaned,pH_cleaned,Rate of Change (ΔT/Δt),Moving Average Deviation,Short-Term Gradient (ΔNTU),Rolling Variance (σ²ₚₕ),temp_fault,tds_fault,turbidity_fault,ph_fault,fault detection
0,2025-12-03 22:45:00+08:00,28.0,252.1596,4,7.535385,NaN,NaN,NaN,NaN,False,False,False,False,[]
1,2025-12-03 23:00:00+08:00,28.0,252.1596,4,7.535385,0.0,NaN,0.0,NaN,False,False,False,False,[]
2,2025-12-03 23:15:00+08:00,28.0,252.1596,4,7.535385,0.0,NaN,0.0,NaN,False,False,False,False,[]
3,2025-12-03 23:30:00+08:00,28.0,252.1596,4,7.535385,0.0,NaN,0.0,NaN,False,False,False,False,[]
4,2025-12-03 23:45:00+08:00,28.0,252.1596,4,7.535385,0.0,0.0,0.0,0.0,False,False,False,False,[]


In [ ]:
import ast
import pandas as pd

def clean_fault_detection(x):
    # Case 1: NaN → empty list
    if pd.isna(x):
        return []

    # Case 2: already a list → keep
    if isinstance(x, list):
        return x

    # Case 3: string → try to parse
    if isinstance(x, str):
        x = x.strip()
        if x == '' or x == '[]':
            return []
        try:
            parsed = ast.literal_eval(x)
            return parsed if isinstance(parsed, list) else []
        except (ValueError, SyntaxError):
            return []

    # Fallback: anything weird
    return []

df['fault detection'] = df['fault detection'].apply(clean_fault_detection)


In [ ]:
if 'fault detection' in df.columns:
    df_filtered = df[
        df['fault detection'].astype(str).str.strip() == '[]'
    ].copy()
else:
    df_filtered = df.copy()

display(df_filtered.head())


,time,temperature_cleaned,tds_cleaned,turbidity_cleaned,pH_cleaned,Rate of Change (ΔT/Δt),Moving Average Deviation,Short-Term Gradient (ΔNTU),Rolling Variance (σ²ₚₕ),temp_fault,tds_fault,turbidity_fault,ph_fault,fault detection
0,2025-12-03 22:45:00+08:00,28.0,252.1596,4,7.535385,NaN,NaN,NaN,NaN,False,False,False,False,[]
1,2025-12-03 23:00:00+08:00,28.0,252.1596,4,7.535385,0.0,NaN,0.0,NaN,False,False,False,False,[]
2,2025-12-03 23:15:00+08:00,28.0,252.1596,4,7.535385,0.0,NaN,0.0,NaN,False,False,False,False,[]
3,2025-12-03 23:30:00+08:00,28.0,252.1596,4,7.535385,0.0,NaN,0.0,NaN,False,False,False,False,[]
4,2025-12-03 23:45:00+08:00,28.0,252.1596,4,7.535385,0.0,0.0,0.0,0.0,False,False,False,False,[]


In [ ]:
from sklearn.ensemble import IsolationForest

# Initialize the Isolation Forest model with specified parameters
isolation_forest_model_overall = IsolationForest(contamination=0.30, random_state=42)


# Fit the model to the df_filtered DataFrame
isolation_forest_model_overall.fit(df_filtered.select_dtypes(include=['number']))

# Predict anomaly labels (-1 for outliers, 1 for inliers)
anomaly_labels_overall = isolation_forest_model_overall.predict(df_filtered.select_dtypes(include=['number']))

# Add anomaly labels to df_filtered
df_filtered['anomaly_label_overall'] = anomaly_labels_overall
df_filtered['anomaly_description_overall'] = df_filtered['anomaly_label_overall'].map({1: 'Inlier', -1: 'Outlier'})

# Display the count of outliers and inliers
anomaly_counts_overall = df_filtered['anomaly_description_overall'].value_counts()
print("Overall Anomaly Detection Summary:")
print(anomaly_counts_overall)

display(df_filtered.head())

Overall Anomaly Detection Summary:
anomaly_description_overall
Inlier     920
Outlier    394
Name: count, dtype: int64


,time,temperature_cleaned,tds_cleaned,turbidity_cleaned,pH_cleaned,Rate of Change (ΔT/Δt),Moving Average Deviation,Short-Term Gradient (ΔNTU),Rolling Variance (σ²ₚₕ),temp_fault,tds_fault,turbidity_fault,ph_fault,fault detection,anomaly_label_overall,anomaly_description_overall
0,2025-12-03 22:45:00+08:00,28.0,252.1596,4,7.535385,NaN,NaN,NaN,NaN,False,False,False,False,[],1,Inlier
1,2025-12-03 23:00:00+08:00,28.0,252.1596,4,7.535385,0.0,NaN,0.0,NaN,False,False,False,False,[],1,Inlier
2,2025-12-03 23:15:00+08:00,28.0,252.1596,4,7.535385,0.0,NaN,0.0,NaN,False,False,False,False,[],1,Inlier
3,2025-12-03 23:30:00+08:00,28.0,252.1596,4,7.535385,0.0,NaN,0.0,NaN,False,False,False,False,[],1,Inlier
4,2025-12-03 23:45:00+08:00,28.0,252.1596,4,7.535385,0.0,0.0,0.0,0.0,False,False,False,False,[],1,Inlier


# Task
Merge the anomaly detection results from the `df_filtered` DataFrame, specifically the 'anomaly_label_overall' and 'anomaly_description_overall' columns, back into the original `df` DataFrame, ensuring all original rows are present. Then, export this combined DataFrame, which includes the anomaly labels for the relevant rows and NaN for others, to a new CSV file named "full_dataset_with_merged_anomalies.csv". Finally, confirm the successful export of this combined dataset.

## Merge Anomaly Results

### Subtask:
Merge the 'anomaly_label_overall' and 'anomaly_description_overall' columns from the 'df_filtered' DataFrame back into the original 'df' DataFrame.


**Reasoning**:
The subtask requires merging anomaly results from `df_filtered` back into `df`. Since `df_filtered` was created by filtering `df` and retains the original index, a left merge on the index is the most robust and efficient way to add the `anomaly_label_overall` and `anomaly_description_overall` columns to `df`, ensuring all original rows are retained and non-matching rows receive NaN values for the new columns.



In [ ]:
anomaly_cols_to_add = df_filtered[['anomaly_label_overall', 'anomaly_description_overall']]

# Perform a left merge on the index to add these columns to the original df
# Rows in df that were not in df_filtered will get NaN for the new columns.
df = df.merge(anomaly_cols_to_add, left_index=True, right_index=True, how='left')

# Display the first few rows of the updated df to verify
display(df.head())

,time,temperature_cleaned,tds_cleaned,turbidity_cleaned,pH_cleaned,Rate of Change (ΔT/Δt),Moving Average Deviation,Short-Term Gradient (ΔNTU),Rolling Variance (σ²ₚₕ),temp_fault,tds_fault,turbidity_fault,ph_fault,fault detection,anomaly_label_overall,anomaly_description_overall
0,2025-12-03 22:45:00+08:00,28.0,252.1596,4,7.535385,NaN,NaN,NaN,NaN,False,False,False,False,[],1.0,Inlier
1,2025-12-03 23:00:00+08:00,28.0,252.1596,4,7.535385,0.0,NaN,0.0,NaN,False,False,False,False,[],1.0,Inlier
2,2025-12-03 23:15:00+08:00,28.0,252.1596,4,7.535385,0.0,NaN,0.0,NaN,False,False,False,False,[],1.0,Inlier
3,2025-12-03 23:30:00+08:00,28.0,252.1596,4,7.535385,0.0,NaN,0.0,NaN,False,False,False,False,[],1.0,Inlier
4,2025-12-03 23:45:00+08:00,28.0,252.1596,4,7.535385,0.0,0.0,0.0,0.0,False,False,False,False,[],1.0,Inlier


## Export Combined Data

### Subtask:
Export the combined DataFrame, which now contains all original rows and the anomaly labels where applicable, to a new CSV file named 'full_dataset_with_merged_anomalies.csv'.


**Reasoning**:
The subtask requires exporting the `df` DataFrame to a CSV file named 'full_dataset_with_merged_anomalies.csv', ensuring the index is not written to the file.



In [ ]:
df.to_csv('full_dataset_with_merged_anomalies.csv', index=False)
print("DataFrame successfully exported to 'full_dataset_with_merged_anomalies.csv'")

DataFrame successfully exported to 'full_dataset_with_merged_anomalies.csv'


# Task
Filter the `df_filtered` DataFrame to isolate only the rows identified as 'Outlier' based on the 'anomaly_description_overall' column, and then export the DataFrame containing only the outliers to a new CSV file named 'outliers.csv', ensuring the index is not written to the file.

## Filter outliers

### Subtask:
Filter the `df_filtered` DataFrame to isolate only the rows identified as 'Outlier' based on the 'anomaly_description_overall' column.


**Reasoning**:
Filter the 'df_filtered' DataFrame to include only rows where 'anomaly_description_overall' is 'Outlier', storing the result in 'df_outliers', and then display the head of 'df_outliers' to verify the filtering.



In [ ]:
df_outliers = df_filtered[df_filtered['anomaly_description_overall'] == 'Outlier']

display(df_outliers.head())

,time,temperature_cleaned,tds_cleaned,turbidity_cleaned,pH_cleaned,Rate of Change (ΔT/Δt),Moving Average Deviation,Short-Term Gradient (ΔNTU),Rolling Variance (σ²ₚₕ),temp_fault,tds_fault,turbidity_fault,ph_fault,fault detection,anomaly_label_overall,anomaly_description_overall
41,2025-12-04 09:00:00+08:00,26.8125,253.4938,4,7.530550,-0.001319,1.06736,0.0,0.000005,False,False,False,False,[],-1,Outlier
136,2025-12-05 08:30:00+08:00,26.1250,258.5891,4,7.516044,-0.001458,6.54240,0.0,0.002473,False,False,False,False,[],-1,Outlier
426,2025-12-08 08:30:00+08:00,25.8750,257.8495,4,7.525714,-0.000347,4.67520,0.0,0.000005,False,False,False,False,[],-1,Outlier
659,2025-12-11 18:00:00+08:00,27.2500,256.6040,3,7.545055,0.001806,-1.06280,-1.0,0.000187,False,False,False,False,[],-1,Outlier
1107,2025-12-18 08:45:00+08:00,25.5625,261.2774,3,7.530550,-0.001875,3.73872,0.0,0.000005,False,False,False,False,[],-1,Outlier


**Reasoning**:
The subtask requires exporting the `df_outliers` DataFrame to a CSV file named 'outliers.csv', ensuring the index is not written to the file.



In [ ]:
df_outliers.to_csv('outliers.csv', index=False)
print("DataFrame 'outliers.csv' successfully exported.")

DataFrame 'outliers.csv' successfully exported.


## Summary:

### Data Analysis Key Findings
*   The `df_filtered` DataFrame was successfully filtered to create `df_outliers`, containing only rows where the 'anomaly\_description\_overall' column is 'Outlier'.
*   The `df_outliers` DataFrame was successfully exported to a CSV file named `outliers.csv` without including the DataFrame index.

### Insights or Next Steps
*   Further analysis can now be performed on the `outliers.csv` file to understand the characteristics and potential causes of these anomalies.


In [ ]:
def append_normal_to_inlier_fault_detection(row):
    if row['anomaly_description_overall'] == 'Inlier':
        # Ensure the value is a list
        if isinstance(row['fault detection'], list):
            # Append 'normal' only if it's not already present
            if 'normal' not in row['fault detection']:
                row['fault detection'].append('normal')
        else:
            # This case should ideally not happen if clean_fault_detection ran
            # but as a safeguard, initialize as a list with 'normal'
            row['fault detection'] = ['normal']
    return row['fault detection']

# Apply the function to the DataFrame
df['fault detection'] = df.apply(append_normal_to_inlier_fault_detection, axis=1)

# Display some rows to verify the changes
display(df[['fault detection', 'anomaly_description_overall']].head())

,fault detection,anomaly_description_overall
0,[normal],Inlier
1,[normal],Inlier
2,[normal],Inlier
3,[normal],Inlier
4,[normal],Inlier


In [ ]:
df.to_csv('full_dataset_with_normal_inliers.csv', index=False)
print("DataFrame successfully exported to 'full_dataset_with_normal_inliers.csv'")

DataFrame successfully exported to 'full_dataset_with_normal_inliers.csv'
